In [ ]:
# apply doc2vec (using title, description columns)
# tried to cluster job titles

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [ ]:
import pandas as pd

file_name = 'hidden'
data = pd.read_csv(file_name, low_memory = False, index_col = 0)

In [ ]:
documents = {}

def extract_titles (row):
    global documents
    if row['description'] and row['description'] == row['description']:
        documents[row['title']] = text_to_word_sequence(row['description'])

In [ ]:
data.apply(extract_titles, axis=1)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# from title_analysis_keras.ipynb
real_symbol_list = ['ş', '8', '₿', '•', '·', 'z', '\xad', '5', 'r', 'p', 'o', 'n', '◆', '►', '🔎', 'g', '1', '4', '6', '9', 'y', '●', '∞', '\xa0', 'f', "'", 'u', 'ר', '0', '⇉', '−', 'm', '\u200e', 'à', '»', '🤫', '＆', '¦', 'k', 'b', '\x96', 'l', 'i', '➟', '✦', 't', '➢', '7', '§', 'в', '‐', '➲', 'j', '★', '\u202c', 'd', '\u200b', 'c', 'e', '◊', '—', '«', 'w', '∙', '\x03', 'v', 'ι', '↔', '♛', '⇒', '♦', '3', '➣', '™', '│', 'и', '–', 'h', 'q']

from os import remove
from nltk.corpus import stopwords

word_list = []
stop_words = set(stopwords.words('english'))

for key, description in documents.items():
    new_description = []
    for word in description:
        if word not in stop_words and word not in real_symbol_list:
            new_description.append(word)
        documents[key] = new_description

In [ ]:
titles = []
for title, description in documents.items():
    titles.append(title)

indexed_titles = []
for i in range(len(titles)):
    new_title = str(i) + "_" + titles[i]
    indexed_titles.append(new_title)

In [ ]:
train = [TaggedDocument(words = documents[titles[i]], tags = titles[i]) for i in range(len(titles))]

In [ ]:
model = Doc2Vec(vector_size = 10, window = 3, epochs = 40, min_count = 1)
model.build_vocab(train)
model.train(train, total_examples=len(train), epochs=50)

In [ ]:
inferreds = []

for i in range(len(titles)):
    inferred = model.infer_vector(documents[titles[i]])
    inferreds.append(inferred)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []
for i in range(1, 11):
    model = KMeans(n_clusters=i)
    model.fit(inferreds)
    inertia.append(model.inertia_)

plt.plot(range(1, 11), inertia)
plt.show()

In [ ]:
n = 2

clustering = KMeans(n_clusters=n)
clustering.fit(inferreds)

In [ ]:
cluster_dict = {i:[] for i in range(0, n)}
for i in range(len(titles)):
    cluster_dict[clustering.labels_[i]].append(titles[i])

cluster_dict